In [17]:
#aula 8 na íntegra: (Criação de Cópia e Escolha do Modelo)
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC  # Import for Support Vector Classifier
from sklearn.metrics import classification_report

# Carregar dados
data = pd.read_csv('/content/a1-in.csv')
data = data.dropna().drop_duplicates()

# Selecionando as variáveis relevantes conforme a tarefa
features = data[['Gender', 'GRADE', 'Activity', 'Transitions/Durations']]
target = data['ONTASK']

# Conversão de 'ONTASK' para binário
target = target.map({'Y': 1, 'N': 0})

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Pipeline para tratamento numérico e categórico
numeric_features = ['GRADE', 'Transitions/Durations']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['Gender', 'Activity']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combinar transformadores
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Modelos a serem avaliados
classifiers = [
    ('logreg', LogisticRegression(max_iter=1000)),
    ('rf', RandomForestClassifier()),
    ('dt', DecisionTreeClassifier()),
    ('svc', SVC())
]

# Grids de hiperparâmetros para GridSearchCV
param_grids = {
    'logreg': {'classifier__C': [0.1, 1, 10]},
    'rf': {'classifier__n_estimators': [50, 100, 200]},
    'dt': {'classifier__max_depth': [None, 10, 20, 30]},
    'svc': {'classifier__C': [0.1, 1, 10]}
}

best_models = {}
for name, classifier in classifiers:
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', classifier)])
    # GridSearchCV para encontrar os melhores parâmetros
    grid_search = GridSearchCV(pipeline, param_grids[name], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"Melhor modelo para {name}: {grid_search.best_params_}")

for name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f"Relatório de classificação para o modelo {name}:")
    print(classification_report(y_test, y_pred))

Melhor modelo para logreg: {'classifier__C': 0.1}
Melhor modelo para rf: {'classifier__n_estimators': 200}
Melhor modelo para dt: {'classifier__max_depth': 10}
Melhor modelo para svc: {'classifier__C': 10}
Relatório de classificação para o modelo logreg:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1790
           1       0.68      1.00      0.81      3757

    accuracy                           0.68      5547
   macro avg       0.34      0.50      0.40      5547
weighted avg       0.46      0.68      0.55      5547



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Relatório de classificação para o modelo rf:
              precision    recall  f1-score   support

           0       0.56      0.19      0.28      1790
           1       0.71      0.93      0.80      3757

    accuracy                           0.69      5547
   macro avg       0.63      0.56      0.54      5547
weighted avg       0.66      0.69      0.63      5547

Relatório de classificação para o modelo dt:
              precision    recall  f1-score   support

           0       0.57      0.16      0.25      1790
           1       0.70      0.94      0.80      3757

    accuracy                           0.69      5547
   macro avg       0.63      0.55      0.52      5547
weighted avg       0.66      0.69      0.62      5547

Relatório de classificação para o modelo svc:
              precision    recall  f1-score   support

           0       0.72      0.01      0.03      1790
           1       0.68      1.00      0.81      3757

    accuracy                           0.68   

## Análise de Melhor Modelo

Após a avaliação de diversos modelos de classificação binária, o modelo de **Floresta Aleatória (Random Forest)** se destacou como o mais promissor para a tarefa de prever comportamentos on-task e off-task em ambientes educacionais. Os resultados obtidos foram comparados baseando-se em métricas de precisão, recall e F1-score para ambas as classes (on-task e off-task).

### Resultados do Modelo de Floresta Aleatória:
- **Precisão para on-task (1):** 71%
- **Recall para on-task (1):** 93%
- **F1-score para on-task (1):** 80%
- **Precisão para off-task (0):** 57%
- **Recall para off-task (0):** 18%
- **F1-score para off-task (0):** 27%

### Justificativa para a Escolha:
O modelo de Floresta Aleatória demonstrou um equilíbrio relativamente melhor entre precisão e recall para ambas as classes em comparação com os outros modelos testados. Apesar de os valores de recall e F1-score para a classe off-task ainda serem baixos, este modelo conseguiu atingir uma precisão e F1-score consideráveis para a classe on-task, que compõe a maioria dos casos.

### Importância das Métricas:
O **recall** é crucial, especialmente para a classe off-task, pois indica a capacidade do modelo de capturar todos os casos relevantes de desengajamento. Um alto recall na classe off-task significa que o sistema é capaz de identificar eficientemente os alunos que precisam de atenção adicional, possibilitando intervenções educacionais mais eficazes.

O **F1-score** também é uma métrica importante, pois equilibra a precisão e o recall, sendo particularmente útil em situações com classes desbalanceadas como esta. O F1-score mais alto para on-task mostra que o modelo é competente em identificar o engajamento dos alunos, mas é necessário melhorar o F1-score para off-task para tornar o modelo mais útil em ambientes educacionais práticos.


--------------------------------------------------------------

In [8]:
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA

##Criação de Nova Feature

# Usando uma amostra menor para testes iniciais (50% dos dados)
data_sample = data.sample(frac=0.5, random_state=42)

# Engenharia de features:
data_sample['Interaction'] = data_sample['NumACTIVITIES'] * data_sample['NumFORMATS']

# Pré-processamento:
features = data_sample.drop('ONTASK', axis=1)
target = data_sample['ONTASK'].map({'Y': 1, 'N': 0})
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Transformações e Feature Engineering:
numeric_features = features.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = features.select_dtypes(include=['object']).columns.tolist()

#Treinamento e Avaliação com GridSearchCV e Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=2))  # Redução de dimensionalidade como exemplo de engenharia de features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

#Implementação do RFE
# Modelo e Seleção de Features com modelo mais leve para RFE: (2.5)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('feature_selection', RFE(estimator=LogisticRegression(), n_features_to_select=5)),
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=10))
])

# Configuração do GridSearchCV:
param_grid = {
    'feature_selection__n_features_to_select': [5, 10]
}

#(2.6)
search = GridSearchCV(model, param_grid, cv=3, n_jobs=-1)
search.fit(X_train, y_train)
print("Melhores parâmetros encontrados:", search.best_params_)

# Avaliação:
y_pred = search.predict(X_test)
print(classification_report(y_test, y_pred))


Melhores parâmetros encontrados: {'feature_selection__n_features_to_select': 10}
              precision    recall  f1-score   support

           0       0.54      0.08      0.14       859
           1       0.70      0.97      0.81      1915

    accuracy                           0.69      2774
   macro avg       0.62      0.52      0.48      2774
weighted avg       0.65      0.69      0.60      2774



In [11]:
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

# Suponhamos que temos um DataFrame chamado `data` já carregado
features = data[['NumACTIVITIES', 'NumFORMATS']]  # Exemplo de seleção de algumas features numéricas

# Criação do objeto PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)

# Fit e transform para criar novas features polinomiais
features_poly = poly.fit_transform(features)

# Convertendo as features polinomiais de volta para um DataFrame
features_poly_df = pd.DataFrame(features_poly, columns=poly.get_feature_names_out(features.columns))

# Exibindo as novas features criadas
print(features_poly_df.head())


   NumACTIVITIES  NumFORMATS  NumACTIVITIES^2  NumACTIVITIES NumFORMATS  \
0            4.0         2.0             16.0                       8.0   
1            4.0         2.0             16.0                       8.0   
2            4.0         2.0             16.0                       8.0   
3            4.0         2.0             16.0                       8.0   
4            4.0         2.0             16.0                       8.0   

   NumFORMATS^2  
0           4.0  
1           4.0  
2           4.0  
3           4.0  
4           4.0  


## Explicação da Feature Criada (2.2)
### Comentário sobre Resultados no Notebook / Reavaliação Pós-RFE

A razão pela qual essa feature foi criada utilizando **Polynomial Features** é baseada na possibilidade de revelar relações complexas e não-lineares entre as variáveis existentes que modelos lineares simples podem não conseguir capturar.

### Benefício

1. **Melhor Desempenho do Modelo**: Modelos que incluem essas novas features podem ter um desempenho melhor, pois estão equipados para entender a complexidade dos dados de forma mais eficaz.

Essa técnica é apenas uma entre várias que podem ser utilizadas para enriquecer o conjunto de dados e potencialmente melhorar os modelos de predição em tarefas de classificação ou regressão.


-----------------------------------------------

# Análise dos resultados:

## Resultados da Aula 8 (2.4)

- **Modelos Utilizados**:
  - Regressão Logística (LogReg)
  - Floresta Aleatória (RF)
  - Árvore de Decisão (DT)
  - Máquinas de Vetores de Suporte (SVC)

- **Desempenho Geral**:
  - Os modelos mostraram uma tendência de precisão alta para a classe positiva (1).

## Resultados da Aula 10

Após a implementação de melhorias, incluindo engenharia de features e seleção de features via `GridSearchCV` e `RFE`, os resultados foram:

- **Melhores parâmetros encontrados**: `{'feature_selection__n_features_to_select': 10}`
- **Métricas de Desempenho**:
  - precision recall f1-score support
  - 0 0.54 0.08 0.14 859
  - 1 0.70 0.97 0.81 1915

- **Accuracy**: 69%
- **Macro avg**:
  - Precision: 0.62
  - Recall: 0.52
  - F1-score: 0.48
- **Weighted avg**:
  - Precision: 0.65
  - Recall: 0.69
  - F1-score: 0.60

## Análise Comparativa (2.7) / Análise Crítica dos Resultados Finais

- **Complexidade vs. Desempenho**: A introdução de engenharia de features e a seleção focada ajudaram a potencialmente simplificar o modelo, apesar de o desempenho em termos de precisão e `recall` ter se mantido similar.
- **Desafios Continuados**: O `recall` dessa classe ainda é baixo.
- **Benefícios da Seleção de Features**: A Aula 10 focou em refinar o modelo para torná-lo mais gerenciável e potencialmente mais explicável, o que é benéfico para entender o impacto das variáveis no engajamento dos alunos.


## Conclusão

Ao analisar criticamente os resultados finais dos modelos das aulas 8 e 10, fica evidente que, apesar das melhorias implementadas na Aula 10, como engenharia de features e seleção de features via RFE (Recursive Feature Elimination), o aumento no desempenho geral do modelo foi baixo. Os modelos continuam a apresentar uma tendência de alta precisão e recall para a classe positiva (engajados), mas lutam significativamente com a classe negativa (não engajados), evidenciado pelo baixo recall nesta categoria. Essa dificuldade em identificar corretamente os alunos desengajados é uma limitação crítica, pois sugere que o modelo, embora eficaz em capturar a maioria dos casos de engajamento, falha em alertar sobre os casos de desengajamento, que são crucialmente importantes para intervenções pedagógicas.

Essa limitação pode ser atribuída à natureza dos dados ou à seleção de features que, apesar do processo de otimização, pode ainda não estar capturando as nuances que diferenciam eficazmente ambas as classes. Também pode indicar a necessidade de revisitar o balanceamento das classes nos dados de treinamento ou de explorar modelos ou métodos de aprendizado de máquina alternativos que possam lidar melhor com desequilíbrios de classe. A implementação de técnicas como o ajuste de pesos de classe no modelo de RandomForest ou a experimentação com algoritmos mais sensíveis a desequilíbrios poderia potencialmente melhorar o desempenho na previsão de alunos desengajados.

Portanto, enquanto a Aula 10 fez progressos no refinamento do modelo, ainda há um caminho significativo a ser explorado na melhoria da capacidade do modelo de atuar de forma eficaz em todas as frentes, o que é essencial para seu uso prático em ambientes educacionais onde identificar ambos, engajamento e desengajamento, é importante.